In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.3 MB/s 
     |████████████████████████████████| 182 kB 38.2 MB/s 
     |████████████████████████████████| 7.6 MB 37.0 MB/s 


In [5]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import os
import io

In [8]:
from google.colab import files
uploaded = files.upload()

Saving ha_train.tsv to ha_train.tsv


In [9]:
uploaded2 = files.upload()

Saving ha_test.tsv to ha_test.tsv


In [10]:
uploaded3 = files.upload()

Saving ha_dev.tsv to ha_dev.tsv


In [14]:
ha_train = pd.read_csv(io.BytesIO(uploaded['ha_train.tsv']),sep='\t')

In [17]:
ha_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9920 entries, 0 to 9919
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    9920 non-null   object
 1   label   9920 non-null   object
dtypes: object(2)
memory usage: 155.1+ KB


In [20]:
ha_train.head()

,text,label
0,@user Kuma dai BBC da rasa aikin yi 🤔😂,neutral
1,@user A kasar uwarwa kuke tweeting ynzu🇳🇬,negative
2,@user Toh mai lampard yakeyi a wajan?💁🏼‍♀️,neutral
3,@user Allah Mai iko dama kina jin 🇮🇳 india to ...,positive
4,@user Agaskiya baa kyautawa mutanen kano ba.😀😀...,negative


In [15]:
ha_test = pd.read_csv(io.BytesIO(uploaded2['ha_test.tsv']),sep='\t')

In [18]:
ha_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835 entries, 0 to 2834
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2835 non-null   object
 1   label   2835 non-null   object
dtypes: object(2)
memory usage: 44.4+ KB


In [21]:
ha_test.head()

,text,label
0,@user Yah Hayyu yah kayyum....Allah ka dube mu...,positive
1,@user @user Ki godewa Allah daya baki Jaybee 😹,positive
2,@user A!! Babbar harka ce wannan amma ni nawa ...,neutral
3,@user @user Allah y Kara taimakun ka 🙏🏽🙏💪 our ...,positive
4,@user Mutane sunji bala'i ta ko ina zasuyi hij...,negative


In [16]:
ha_dev = pd.read_csv(io.BytesIO(uploaded3['ha_dev.tsv']),sep='\t')

In [19]:
ha_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1417 entries, 0 to 1416
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1417 non-null   object
 1   label   1417 non-null   object
dtypes: object(2)
memory usage: 22.3+ KB


In [22]:
ha_dev.head()

,text,label
0,@user Wani farfesu ne naci mai karni da yawa.....,negative
1,@user Gsky ALLAAH yakara kareka Gov'n Ganduje ...,positive
2,@user Mudae munama fatan alkhairi adalin shuga...,positive
3,@user Tun muna da Hope bamu baka 1k din bane s...,negative
4,@user To shi Allah ina ruwansa🤷‍♂️,neutral


In [3]:
model = "Davlan/naija-twitter-sentiment-afriberta-large"  # works for ha as well as 3 other languages
tokenizer = AutoTokenizer.from_pretrained(model)
# HA
model_ha = AutoModelForSequenceClassification.from_pretrained(model)

Downloading:   0%|          | 0.00/429 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/503M [00:00<?, ?B/s]

In [23]:
def predict_by_ntsal(text):
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model_ha(**encoded_input) 
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  id2label = {0:"positive", 1:"neutral", 2:"negative"}

  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  predicted_label = id2label[ranking[0]]
  return predicted_label

In [24]:
# evaluation method

def evaluate(data_and_labels):
  #data_and_labels = pd.read_csv(file_path, sep='\t') 
  data = data_and_labels.text
  trues = data_and_labels.label 
  preds = []
  l = list(trues)
  for review in data: 
    predicted_label = predict_by_ntsal(review)
    preds.append(predicted_label)
    
    
  return classification_report(l, preds)

In [ ]:
# train test split


In [25]:
print('Ha Language Train Results')
print(evaluate(ha_train))

Ha Language Train Results
              precision    recall  f1-score   support

    negative       0.95      0.94      0.94      3254
     neutral       0.93      0.94      0.94      3452
    positive       0.96      0.97      0.97      3214

    accuracy                           0.95      9920
   macro avg       0.95      0.95      0.95      9920
weighted avg       0.95      0.95      0.95      9920



In [26]:
print('Ha Language Test Results')
print(evaluate(ha_test))

Ha Language Test Results
              precision    recall  f1-score   support

    negative       0.96      0.94      0.95       889
     neutral       0.94      0.96      0.95       980
    positive       0.97      0.97      0.97       966

    accuracy                           0.96      2835
   macro avg       0.96      0.96      0.96      2835
weighted avg       0.96      0.96      0.96      2835



In [27]:
print('Ha Language Dev Results')
print(evaluate(ha_dev))

Ha Language Dev Results
              precision    recall  f1-score   support

    negative       0.94      0.93      0.94       430
     neutral       0.93      0.94      0.94       480
    positive       0.98      0.97      0.98       507

    accuracy                           0.95      1417
   macro avg       0.95      0.95      0.95      1417
weighted avg       0.95      0.95      0.95      1417

